In [1]:
# Import Dependencies 

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup 

import pymongo

import requests

from splinter import Browser

import pandas as pd

In [2]:
#Chromedriver

executable_path = {'executable_path': ChromeDriverManager().install()}

browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\taylo\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [3]:
# Begin News url scrape

news_url = 'https://redplanetscience.com'

browser.visit(news_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Find most recent article 
results = soup.find_all('div',class_="list_text")

news_title = results[0].find('div',class_='content_title').text

news_p = results[0].find('div',class_='article_teaser_body').text

print(news_title)
print('----')
print(news_p)

While Stargazing on Mars, NASA's Curiosity Rover Spots Earth and Venus
----
This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.


In [5]:
# Begin jpl scrape

jpl_url = 'https://spaceimages-mars.com'

browser.visit(jpl_url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Get image url 
jpl_scrape_url = 'https://spaceimages-mars.com/'

img_url = soup.find_all('a',class_='showimg fancybox-thumbs')[0]['href']

featured_url = jpl_scrape_url+img_url

print(featured_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [7]:
#Galaxy facts scrape
galaxy_facts_url = 'https://galaxyfacts-mars.com'

mars_info = pd.read_html(galaxy_facts_url)

planet_profile = mars_info[1]

mars_html = planet_profile.to_html(index=False, header=False)

print(mars_html)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [8]:
# Mars Hemispheres Scrape.

mars_hemi_url = 'https://marshemispheres.com/'

browser.visit(mars_hemi_url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [9]:
#find urls and createw empty list to hold them
hemi_url = soup.find_all('div', class_='description')
hemi_images = []
hemi_names = ["Ceberus", "Schiaparelli", "Syrtis Major", "Valles Marineris"]



In [17]:
for hemi_urls in hemi_url:
    
    hemi_page = hemi_url.find('a', class_='itemLink')['href']
    
    hemi_page = f'{mars_hemi_url}{hemi_page}'

    hemi_response = requests.get(hemi_page)
    
    soup2 = BeautifulSoup(hemi_response.text, 'lxml')
    
    hemi_img_soup = soup2.find_all('li')[0]
    
    hemi_img = hemi_img_soup.find('a')['href']
    
    hemi_img = f'{mars_hemi_url}{hemi_img}'
    
    hemi_images.append(hemi_img)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [13]:
hemi_images

['https://marshemispheres.com/images/full.jpg',
 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg']

In [14]:
hemisphere_image_urls = [
    {"title": "Cerberus", "img_url": hemi_images[0]},
    {"title": "Schiaparelli", "img_url": hemi_images[1]},
    {"title": "Syrtis Major", "img_url": hemi_images[2]},
    {"title": "Valles Marineris", "img_url": hemi_images[3]}
]
hemisphere_image_urls

[{'title': 'Cerberus',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [15]:
mars_dict = {
    "article_title": news_title,
    "article_teaser": news_p,
    "image": featured_url,
    "table": mars_html,
    "hemisphere_names": hemi_names,
    "hemispheres_dict":hemisphere_image_urls
    
}

In [16]:
mars_dict

{'article_title': "While Stargazing on Mars, NASA's Curiosity Rover Spots Earth and Venus",
 'article_teaser': "This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.",
 'image': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'table': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\